# まずはライブラリをインポート

In [1]:
import transformers
import torch

# 最大文字数を指定
文ごとに文字数が変わるが、学習・推論する時には一定サイズのベクトルを渡す必要がある。  
そのため、最大文字数を決めておく必要がある。  
今回は192

In [2]:
MAX_LENGTH = 192

# エンコードする文字ベクトルを作成
詳しくは
[ここ](https://huggingface.co/transformers/model_doc/roberta.html)  
「This is a pen.」と「I am a man」を1つの文章として、トークンIDへ変換。

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("roberta-base")
text = "This is a pen."
text2 = "I am a man"

ids = tokenizer.encode(text)
ids2 = tokenizer.encode(text2)

token_ids = tokenizer.build_inputs_with_special_tokens(ids, ids2)
token_ids

[0, 713, 16, 10, 7670, 4, 2, 2, 100, 524, 10, 313, 2]

# パディング＆Attention Maskの作成
文字数の最大値を指定したが、たいていのケースで最大文字数に達しない。  
そのため、使わない領域として、***特別な文字***([参照](https://qiita.com/ishikawa-takumi/items/5fc45ddd121b23db5de9))で埋めてあげる必要がある。  
また、有効な領域だとモデル示すために、Attention maskというものも用意する。  
具体的には、有効な文字があるところが「１」、パティングしたところが「０」となる。  
maskのサイズも文字最大数と同じ。

In [4]:
mask = [1] * len(token_ids)

In [5]:
padding_length = MAX_LENGTH - len(token_ids)
if padding_length > 0:
    token_ids = token_ids + ([1] * padding_length)
    mask = mask + ([0] * padding_length)

# Modelの生成
RoBERTaを利用

In [6]:
model = transformers.AutoModel.from_pretrained("roberta-base")
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

# Modelを使ってエンコード
## まずはIDとmaskをmodelに入力できる型に変換

In [7]:
token_ids_tensor = torch.tensor([token_ids], dtype=torch.long)
mask_tensor = torch.tensor([mask], dtype=torch.long)

## Modelで処理

In [8]:
out = model(input_ids=token_ids_tensor, attention_mask=mask_tensor)
out

(tensor([[[-0.0977,  0.0892, -0.0099,  ..., -0.0847, -0.0366, -0.0534],
          [ 0.0274,  0.2033,  0.1038,  ...,  0.2127, -0.0719, -0.0050],
          [ 0.1462,  0.1765,  0.1418,  ..., -0.2564, -0.0259,  0.0907],
          ...,
          [-0.0685,  0.0695,  0.0897,  ...,  0.0282, -0.0272,  0.0324],
          [-0.0511,  0.0756,  0.0510,  ...,  0.0246, -0.0135,  0.0227],
          [-0.0344,  0.0882,  0.0317,  ...,  0.0632, -0.0208,  0.0009]]],
        grad_fn=<NativeLayerNormBackward>),
 tensor([[-1.0725e-02, -2.2089e-01, -2.0918e-01, -6.8649e-02,  1.2540e-01,
           2.1667e-01,  2.7022e-01, -7.1740e-02, -9.9609e-02, -1.6666e-01,
           2.6844e-01, -1.6258e-02, -1.1163e-01,  1.3448e-01, -1.3481e-01,
           4.8152e-01,  2.2144e-01, -5.0482e-01,  6.7738e-02, -7.9790e-03,
          -2.9393e-01,  6.0414e-02,  4.7367e-01,  3.3544e-01,  1.1456e-01,
           5.4199e-02, -1.6571e-01, -1.8260e-02,  1.6509e-01,  2.5049e-01,
           2.9651e-01,  4.9777e-02,  1.2795e-01,  2.4528e

# outputのサイズ
out[0]:最後のBertLayerの出力 
out[1]:最後のBertLayerの0要素目（CLS）を入力したBertPoolerの出力

In [10]:
print(out[0].shape)
print(out[1].shape)

torch.Size([1, 192, 768])
torch.Size([1, 768])
